In [11]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
import math
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from selenium.webdriver.common.by import By
import json
import pygsheets
from google.oauth2 import service_account
import platform
import datetime
from dateutil.relativedelta import relativedelta
import sys
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

class MyChromeDriver(webdriver.Chrome):

    def __init__(self, headless=False, maximize=True):

        options = webdriver.ChromeOptions()
        if headless:
            options.add_argument('--headless')

        webdriver.Chrome.__init__(self, service=ChromeService(ChromeDriverManager().install()), options=options)

        if maximize:
            self.maximize_window()

os.getcwd()

'c:\\Users\\SANDBOX\\Desktop\\Sandbox\\crawling'

In [12]:
# 게이밍 시트 가져오기 : 시트ID로
gc = pygsheets.authorize(service_account_file='creds.json')

sheet_id = '1w_JkKqp08e_fV_fPJHRKqwS1FXg7jfyy5ycNXDKdwNw'
wks = gc.open_by_key(sheet_id)
sh = wks.worksheet_by_title('크롤링 리스트(23.4Q)')
gaming = sh.get_as_df(has_header=True , index_column=None , start='A1', end='H100000' , numerize=True , empty_value=np.nan)

# 크롤링 기준
gaming_0_1 = gaming[(gaming['게임 광고'] == 0) | (gaming['게임 광고'] == 1)]
gaming_0_1

,채널명,채널ID,구독자,유튜브 링크,게임 광고,중복 체크,분기
148,김블루,UCNzcxCN_Hh_lu5RCSFXKgGQ,"1,740,000",https://www.youtube.com/channel/UCNzcxCN_Hh_lu...,1,중복X,NaN
149,홍방장,UCHxrU6bjWrnBfmReUAeztcw,"272,000",https://www.youtube.com/channel/UCHxrU6bjWrnBf...,1,중복X,NaN
150,침착맨,UCUj6rrhMTR9pipbAWBAMvUQ,"2,260,000",https://www.youtube.com/channel/UCUj6rrhMTR9pi...,1,중복X,NaN
151,오킹,UCom6YhUY62jM52nIMjf5_dw,"2,080,000",https://www.youtube.com/channel/UCom6YhUY62jM5...,1,중복X,NaN
152,김달걀,UCfNUrgI9-Q5ySUJo2VUoX1Q,"81,500",https://www.youtube.com/channel/UCfNUrgI9-Q5yS...,1,중복X,NaN
...,...,...,...,...,...,...,...
591,사나이 김기훈,UC9nFMAQcn0bqIrEV9vUdU1g,"1,160,000",https://www.youtube.com/channel/UC9nFMAQcn0bqI...,0,중복X,NaN
592,라로시 Larothy,UCJXbMRXtX6vHDkGoeerk8Yw,"4,480",https://www.youtube.com/channel/UCJXbMRXtX6vHD...,0,중복X,NaN
593,싸패TV,UCfycsfXKilVd-EBL9VC8RGA,"97,000",https://www.youtube.com/channel/UCfycsfXKilVd-...,0,중복X,NaN
594,블러비BLOVEY,UCXz_coEujrsE9WMJJbg6lTg,"88,900",https://www.youtube.com/channel/UCXz_coEujrsE9...,0,중복X,NaN


In [ ]:
gaming_0_1.to_csv('게이밍 크롤링 리스트')

In [13]:
len(gaming_0_1['채널ID'].unique())

446

In [19]:
import datetime

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
if driver is None:
        my_driver = MyChromeDriver()
        time.sleep(10)
else:
    my_driver = driver
    
channel_id_list = gaming_0_1['채널ID'].unique().tolist()
video_url_list = []
youtube_video_url_dic = {}
    
# 현재 날짜 계산
current_date = datetime.datetime.now()
# 3개월 전 날짜 계산
three_months_ago = current_date - datetime.timedelta(days=90)

for i in tqdm(range(len(channel_id_list))):
    url = 'https://www.youtube.com/channel/' + channel_id_list[i] + '/videos'
    driver.get(url)
    time.sleep(3)
    print(i, url)

    pageString = driver.page_source
    soup = BeautifulSoup(pageString, features='html.parser')

    video_url_list = []  # 비디오 URL을 저장할 리스트 추가
    date_list = ['분 전', '시간 전', '1일 전', '2일 전', '3일 전', '4일 전', '5일 전', '6일 전', '7일 전', '8일 전', '9일 전',
                 '10일 전', '11일 전', '12일 전', '13일 전', '2주 전', '3주 전', '4주 전', '1개월 전', '2개월 전', '3개월 전']

    # 날짜
    date_soup = soup.find_all('span', attrs={'class' : 'inline-metadata-item style-scope ytd-video-meta-block'})[1::2]

    for k in range(len(date_soup)):
        video_date = date_soup[k].text

        # 숫자 부분 추출 (날짜를 정수로 변환하지 않고 그대로 사용)
        numeric_part = ''.join(filter(str.isdigit, video_date))

        # 해당 일자의 비디오 URL 저장
        if numeric_part:
            video_links = soup.find_all('a', {'class': 'yt-simple-endpoint style-scope ytd-grid-video-renderer'})
            if k < len(video_links):
                video_link = video_links[k]['href']
                video_url = 'https://www.youtube.com' + video_link
                video_url_list.append(video_url)  # 비디오 URL 저장
                print("영상 링크:", video_url)


    youtube_video_url_dic[channel_id_list[i]] = video_url_list  # 비디오 URL 저장

driver.quit()


  0%|          | 0/446 [00:00<?, ?it/s]

0 https://www.youtube.com/channel/UCNzcxCN_Hh_lu5RCSFXKgGQ/videos


  0%|          | 1/446 [00:06<51:30,  6.94s/it]

1 https://www.youtube.com/channel/UCHxrU6bjWrnBfmReUAeztcw/videos


  0%|          | 2/446 [00:11<42:34,  5.75s/it]

...

77 https://www.youtube.com/channel/UCj9LQu5-mfrf6imcyrnZfug/videos


 17%|█▋        | 78/446 [07:11<33:10,  5.41s/it]

78 https://www.youtube.com/channel/UCLoGhS_8p-G_pzrkfgMi9NA/videos


 18%|█▊        | 79/446 [07:21<34:10,  5.59s/it]


KeyboardInterrupt: 

In [25]:
soup

<html darker-dark-theme="" darker-dark-theme-deprecate="" lang="ko-KR" style="font-size: 10px;font-family: Roboto, Arial, sans-serif;" system-icons="" typography="" typography-spacing=""><head><script data-original-src="/s/player/2626486d/player_ias.vflset/ko_KR/miniplayer.js" nonce="" src="/s/player/2626486d/player_ias.vflset/ko_KR/miniplayer.js"></script><script data-original-src="/s/player/2626486d/player_ias.vflset/ko_KR/remote.js" nonce="" src="/s/player/2626486d/player_ias.vflset/ko_KR/remote.js"></script><script data-original-src="/s/player/2626486d/player_ias.vflset/ko_KR/offline.js" nonce="" src="/s/player/2626486d/player_ias.vflset/ko_KR/offline.js"></script><script data-id="_gd" nonce="">window.WIZ_global_data = {"MuJWjd":false,"nQyAE":{},"oxN3nb":{"1":false}};</script><meta content="ApvK67ociHgr2egd6c2ZjrfPuRs8BHcvSggogIOPQNH7GJ3cVlyJ1NOq/COCdj0+zxskqHt9HgLLETc8qqD+vwsAAABteyJvcmlnaW4iOiJodHRwczovL3lvdXR1YmUuY29tOjQ0MyIsImZlYXR1cmUiOiJQcml2YWN5U2FuZGJveEFkc0FQSXMiLCJleHBpcn

In [14]:
# 각 채널의 3개월 전 영상 크롤링

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
if driver is None:
        my_driver = MyChromeDriver()
        time.sleep(10)
else:
    my_driver = driver
    
channel_id_list = gaming_0_1['채널ID'].unique().tolist()
video_url_list = []
youtube_video_url_dic = {}

for i in tqdm(range(len(channel_id_list))):
    url = 'https://www.youtube.com/channel/' + channel_id_list[i] + '/videos'
    driver.get(url)
    time.sleep(3)
    print(i, url)

    pageString = driver.page_source
    soup = BeautifulSoup(pageString, features='html.parser')

    date_count = 0
    video_date_list = []
    date_list = ['분 전', '시간 전', '1일 전', '2일 전', '3일 전', '4일 전', '5일 전', '6일 전', '7일 전', '8일 전', '9일 전',
                 '10일 전', '11일 전', '12일 전', '13일 전', '2주 전', '3주 전', '4주 전', '1개월 전', '2개월 전', '3개월 전']
    
    # 날짜
    date_soup = soup.find_all('span', attrs={'class' : 'inline-metadata-item style-scope ytd-video-meta-block'})[1::2]

    # 날짜 기준에 맞는 영상 개수 카운트
    for j in range(len(date_soup)):
        if date_soup[j].text in date_list:
            date_count += 1
        if date_list[0] in date_soup[j].text:
            date_count += 1

    # 날짜 기준에 맞도록 날짜 정보 저장
    date_soup = date_soup[:date_count]
    for k in range(len(date_soup)):
        video_date = date_soup[k].text
        video_date_list.append(video_date)
    
    # 영상 url
    for i in range(len(soup.find_all('ytd-video-renderer',{'class':'style-scope ytd-item-section-renderer'}))): 
    #soup.find_all('ytd-video-meta-block', 'style-scope ytd-video-renderer byline-separated'))):
        if 'shorts' in soup.find_all('a',{'id':'video-title'})[i].get('href'):
            video = 'https://youtu.be/'+ soup.find_all('a',{'id':'video-title'})[i].get('href').split('/')[2]
            video_url_list.append(video)
        else :
            video = 'https://youtu.be/'+ soup.find_all('a',{'id':'video-title'})[i].get('href').split('=')[1]
            video_url_list.append(video)
    
    youtube_video_url_dic[channel_id_list[i]] = video_date_list
    youtube_video_url_dic[channel_id_list[i]] = video_url_list
    
driver.quit()

  0%|          | 0/446 [00:00<?, ?it/s]

0 https://www.youtube.com/channel/UCNzcxCN_Hh_lu5RCSFXKgGQ/videos


  0%|          | 1/446 [00:06<49:21,  6.66s/it]

1 https://www.youtube.com/channel/UCHxrU6bjWrnBfmReUAeztcw/videos


  0%|          | 2/446 [00:12<45:08,  6.10s/it]

2 https://www.youtube.com/channel/UCUj6rrhMTR9pipbAWBAMvUQ/videos


  1%|          | 3/446 [00:21<53:31,  7.25s/it]


KeyboardInterrupt: 

In [23]:
youtube_video_url_dic

{'UCNzcxCN_Hh_lu5RCSFXKgGQ': [],
 'UCHxrU6bjWrnBfmReUAeztcw': [],
 'UCUj6rrhMTR9pipbAWBAMvUQ': [],
 'UCom6YhUY62jM52nIMjf5_dw': [],
 'UCfNUrgI9-Q5ySUJo2VUoX1Q': [],
 'UCXAXCxTCCYYKsLJNzEvjqfw': [],
 'UCTl2piKm6GSzHeYKr-SAopw': [],
 'UCfycCZCzxBrdBzNF1fGD5mA': [],
 'UCmCNgLHdknmCdQelpwxQvvg': [],
 'UC0z8bddfRBRBmEC-AjI6UXw': [],
 'UC-jHB5e-6LF-yz0kMCTZWdg': [],
 'UCmJ4DkPFPzjR2TkJ7lXSQWw': [],
 'UCI6Ro_MOr36re4Bhb7nCY7g': [],
 'UCrktsSopGh4xCdK1WrIKG8g': [],
 'UCHE7GBQVtdh-c1m3tjFdevQ': [],
 'UC3KlqJ3v_43nhvDaXfxFAXw': [],
 'UCAzdzLfHZdN-KnchZB5luqg': [],
 'UCBkyj16n2snkRg1BAzpovXQ': [],
 'UCIchCOLwLBMD87quHlNuXjQ': [],
 'UCIVFv8AiQLqM9oLHTixrNYw': [],
 'UCb0eyf03Jz8Q5j2q65TzCoA': [],
 'UCBA9XaL5wCdHnC5EmEzwrqw': [],
 'UCQk2rWiBjYjTy1Ee83PlpNA': [],
 'UCcSQ_5lbCgf9ruiMG8iBoGg': [],
 'UCArfE1gUOZgyJusznHqXciA': [],
 'UCbFzvzDu17eDZ3RIeaLRswQ': [],
 'UCUY0FVprY69TQu5pbho2PNw': [],
 'UC1L79yKbmVepsCYusa4KHLQ': [],
 'UConOmISJY0oZHRhWGyvdQ2A': [],
 'UCqFt6raBlXK-qJCWsTKW62A': [],
 'UCXOLqHs

In [20]:
len(youtube_video_url_dic)

79

In [22]:
# 채널 ID와 크롤링한 비디오 URL 데이터를 리스트로 저장
gaming_df = []

for channel_id, video_url_list in youtube_video_url_dic.items():
    for video_url in video_url_list:
        gaming_df.append([channel_id, video_url])

# 데이터프레임 생성
gaming_df = pd.DataFrame(gaming_df, columns=['Channel_ID', 'Video_URL'])
gaming_df

,Channel_ID,Video_URL


In [21]:
gaming_video_df = pd.DataFrame()

for i in range(len(youtube_video_url_dic.keys())):
    new_df = pd.DataFrame({'channel id':[list(youtube_video_url_dic.keys())[i]] * len(youtube_video_url_dic[list(youtube_video_url_dic.keys())[i]]), 
                           'url': youtube_video_url_dic[list(youtube_video_url_dic.keys())[i]]})
    gaming_video_df = pd.concat([gaming_video_df, new_df])
    
gaming_video_df = gaming_video_df.reset_index(drop=True)
gaming_video_df

,channel id,url


In [ ]:
# 영상 유료광고 배너 크롤링
driver = webdriver.Chrome()

video_url_list = []
channel_list = []
channel_id_list = []
sub_list = []
ad_banner_list = []

for i in tqdm(range(len(video_list))):
    driver.get(video_list[i])
    time.sleep(3)
    
    try:
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
    
        # 채널ID
        try:
            channel_id = str(soup).split(',"channelId":"')[1].split('","')[0]
        except:
            channel_id = '채널ID 크롤링 오류'
            continue
            
        # 구독자 수
        try:
            sub = soup.find('yt-formatted-string', {'id': 'owner-sub-count'}).text.split('구독자')[1].strip().split('명')[0].strip()
        except:
            sub = '구독자 수 크롤링 오류'
            continue
            
        # 유료 광고 영상 배너
        try:
            ad_banner = soup.find_all('div', class_='ytp-paid-content-overlay-text')[0].text
        except:
            ad_banner = 'X'
            continue
        
        # 모든 데이터가 성공적으로 크롤링되면 리스트에 추가
        video_url_list.append(video_list[i])
        keyword_list_2.append(keyword_list[i])
        title_list.append(title)
        channel_list.append(channel)
        channel_id_list.append(channel_id)
        date_list.append(date)
        view_list.append(view)
        sub_list.append(sub)
        ad_banner_list.append(ad_banner)
        description_list.append(description)

        print(i, video_url_list[-1], keyword_list_2[-1], title_list[-1], channel_list[-1], channel_id_list[-1], date_list[-1], view_list[-1], sub_list[-1], ad_banner_list[-1], description_list[-1])
        
    except:
        print("크롤링 오류")

driver.quit()